### Extracción de Datos
La extracción de datos es un paso fundamental en el proceso ETL. Es importante comprender los diferentes pasos involucrados en la extracción de datos y las herramientas disponibles para realizar esta tarea. Al seguir las mejores prácticas y solucionar los problemas de manera efectiva, puedes asegurarte de que el proceso de extracción de datos se complete con éxito.



### Importamos librerías

Comienza instalando solo las librerías que necesitas para empezar tu proyecto.

In [2]:
import json
import pandas as pd
import glob
import ast
import pyarrow as pa
import pyarrow.parquet as pq

Importación de las rutas de los archivos a trabajar

In [3]:
directorio = "../0-DATA/*.json" # importamos los archivos .json que se encuentran dentro del directorio 0-DATA
archivos_json = glob.glob(directorio)  # busca y guarda en una lista todas las rutas de archivos con extensión .json que se encuentren dentro del directorio especificado.

In [4]:
archivos_json # imprimimos la lista de rutas de los archivos con extensión json

['../0-DATA\\australian_users_items.json',
 '../0-DATA\\australian_user_reviews.json',
 '../0-DATA\\output_steam_games.json']

### La lectura de datos

La lectura de datos es un paso fundamental en el análisis de datos y el aprendizaje automático. Se trata del proceso de cargar datos desde diferentes fuentes, como archivos CSV, archivos de Excel, bases de datos o APIs, en un formato que pueda ser procesado por un programa.


Lectura del archivo **australian_users_items.json**

In [5]:
rows1=[]
with open(archivos_json[0],encoding="utf-8") as file: #lectura del archivo 'australian_users_items.json', modo de lectura encoding="utf-8"
    for line in file.readlines(): # iteramos fila por fila el archivo file
        rows1.append(ast.literal_eval(line)) # guardamos las filas en la lista rows1
items=pd.DataFrame(rows1) # presentamos nuestra lista en un DataFrame

Lectura del archivo **australian_user_reviews.json**

In [5]:
rows2=[]
with open(archivos_json[1],encoding="MacRoman") as file: #lectura del archivo 'australian_users_items.json', modo de lectura encoding="MacRoman"
    for line in file.readlines():
        rows2.append(ast.literal_eval(line)) 
reviews=pd.DataFrame(rows2) 

Lectura del archivo **output_steam_games.json**

In [6]:
rows3=[]
with open(archivos_json[2]) as file: #lectura del archivo 'australian_users_items.json', sin modo de lectura ya que este no tiene parámetros de otro idioma fuera del Inglés
    for line in file.readlines():
        rows3.append(json.loads(line))
games=pd.DataFrame(rows3) 

### Presentación y procesamiento para la extracción de data de los archivos json

Extracción de datos del archivo **australian_users_items.json**

In [7]:
items.head(2) #presentación en un DataFrame los datos del archivo australian_users_items.json, contiene filas anidadas en la columna 'items'

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


In [8]:
#aplicamos el método .info() para ver la información a detalle estructuralmente de nuestros datos
items.info() #podemos visualizar que tenemos 88310 filas x 5 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [9]:
#hacemos una normalización con la funcion .json_normalize() de la librería Pandas para desanidar la columna 'items'
normalizado = pd.json_normalize(rows1, record_path=['items'], #'row1' es la lista de nuestra lectura y 'record_path' indica la columna a desanidar 
                                meta=['steam_id','items_count','user_id', 'user_url'] ) #'meta' indica las columnas que se unirán a las nuevas columnas de los datos desanidados
normalizado.head(2)

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [10]:
#aplicamos el método .info() nuevamente para ver la información de la nueva estructuralmente de nuestros datos
normalizado.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   steam_id          object
 5   items_count       object
 6   user_id           object
 7   user_url          object
dtypes: int64(2), object(6)
memory usage: 314.5+ MB


In [11]:
#realizamos una copia, así continuamos nuestro trabajo sin perder información por mala manipulación de nuestra Data, es buena práctica hacerlo
normalizados = normalizado.copy() # copiamos el dataframe para no modificar el original

In [12]:
#guardamos el último DataFrame un archivo .csv sin anidaciones para proceder después con el ETL respectivo .
normalizados.to_csv("../0-DATA/australian_users_items.csv", index=False, encoding="utf-8") # se guarda en estándar de codificación de caracteres utf-8 para no tener problemas de lectura a futuro.

In [13]:
#Procedemos de la misma manera guardando en un archivo .parquet
itemsl = pd.read_csv("../0-DATA/australian_users_items.csv") # importamos el archivo csv guardado, ya que es mejor pasar de .csv a .parquet estructuralmente
tabla = pa.Table.from_pandas(itemsl) # convertimos el DataFrame en una tabla de formato apache arrow
pq.write_table(tabla,"../0-DATA/australian_users_items.parquet") # guardamos en la ruta nuestra data en archivo .parquet

Extracción de datos del archivo **australian_user_reviews.json**

In [14]:
reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [15]:
reviews.info() #podemos visualizar que tenemos 25799  filas x 3 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [16]:
#hacemos una normalización con la funcion .json_normalize() de la librería Pandas para desanidar la columna 'reviews'
normalizado = pd.json_normalize(rows2, record_path=['reviews'], #'row2' es la lista de nuestra lectura y 'record_path' indica la columna a desanidar 
                                meta=['user_id','user_url'] ) #'meta' indica las columnas que se unirán a las nuevas columnas de los datos desanidados
normalizado.head(2)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [17]:
normalizado.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   funny        59305 non-null  object
 1   posted       59305 non-null  object
 2   last_edited  59305 non-null  object
 3   item_id      59305 non-null  object
 4   helpful      59305 non-null  object
 5   recommend    59305 non-null  bool  
 6   review       59305 non-null  object
 7   user_id      59305 non-null  object
 8   user_url     59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


In [18]:
normalizados = normalizado.copy()

In [19]:
#guardamos el último DataFrame un archivo .csv sin anidaciones para proceder después con el ETL respectivo .
normalizados.to_csv("../0-DATA/australian_user_reviews.csv", index=False, encoding="utf-8") 

In [20]:
#Procedemos de la misma manera guardando en un archivo .parquet
itemsl = pd.read_csv("../0-DATA/australian_user_reviews.csv")
tabla = pa.Table.from_pandas(itemsl) 
pq.write_table(tabla,"../0-DATA/australian_user_reviews.parquet") 

Extracción de datos del archivo **output_steam_games.json**

In [21]:
games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
games.info() #podemos visualizar que tenemos 120445 filas x 13 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [23]:
game = games.copy() 

In [24]:
#guardamos el último DataFrame un archivo .csv sin anidaciones para proceder después con el ETL respectivo .
game.to_csv("../0-DATA/output_steam_games.csv", index=False)#, encoding="utf-8") 

In [25]:
#Procedemos de la misma manera guardando en un archivo .parquet
itemsl = pd.read_csv("../0-DATA/output_steam_games.csv") 
tabla = pa.Table.from_pandas(itemsl) 
pq.write_table(tabla,"../0-DATA/output_steam_games.parquet") 

C:\Users\Pc\AppData\Local\Temp\ipykernel_12124\4083086056.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  itemsl = pd.read_csv("../0-DATA/output_steam_games.csv") # leemos el archivo csv
